### Load Package and Data

In [1]:
import pandas as pd
import joblib
import pandas as pd
import numpy as np
from typing import Tuple, Sequence, List, Optional, Iterable, Any, Iterator, Callable
import textdistance

def levenshtein_similarity(truth: List[str], pred: List[str]) -> float:
    assert len(truth) == len(pred)
    scores = sum(textdistance.levenshtein.normalized_similarity(t, p) for t, p in zip(truth, pred))
    return scores / len(truth)

def full_sentence_accuracy(truth: List[str], pred: List[str]) -> float:
    """Calculate the number of exact matches."""
    assert len(truth) == len(pred)
    correct_count = sum(int(t == p) for t, p in zip(truth, pred))
    return correct_count / len(truth)

truth_file = 'data/data_for_llms/test/test_300.csv'
predict_file = "results/predictions/prediction_of_saved_models-Meta-Llama-3-8B-Instruct-train_200_lr5e-06_bs2-checkpoint-400.csv" # 100-600, 400 best
predict_file = 'results/predictions/prediction_of_saved_models-Mistral-7B-Instruct-v0.2-train_200_lr5e-06_bs2-checkpoint-200.csv' # 100-400, 200 best
predict_file = 'results/predictions/prediction_of_saved_models-Meta-Llama-3-8B-Instruct-train_200_without_prompt_lr5e-06_bs2-checkpoint-400.csv' # 100-400, 400 best
predict_file = 'results/predictions/prediction_of_saved_models-Mistral-7B-Instruct-v0.2-train_200_without_prompt_lr5e-06_bs2-checkpoint-200.csv' # 100-400, 200 best

truth_df = pd.read_csv(truth_file, encoding='utf-8').fillna('N/A')
predict_df = pd.read_csv(predict_file, encoding='utf-8').fillna('N/A')
predict_df

,Actual Text,Paragraph,IUPAC,1H NMR text,1H NMR conditions,1H NMR data,13C NMR text,13C NMR conditions,13C NMR data,19F NMR text,19F NMR conditions,19F NMR data
0,"{""IUPAC"":""1-(3,5-bis(trifluoromethyl)phenyl)-2...",Synthesis and crystallization A stirred soluti...,"1-(3,5-bis-CF3phenyl)-2-bromoethanone","1H NMR: (CDCl3, 600 MHz): 8.44 (2H, s), 8.13 (...","CDCl3, 600 MHz","8.44, 8.13, 4.48","13C NMR: (CDCl3, 150 MHz): 188.81, 135.31, 133...","CDCl3, 150 MHz","188.81, 135.31, 133.06, 132.83, 132.60, 128.99...",N/A,N/A,N/A
1,"{""IUPAC"":""N-(diaminomethylidene)-4-[(E)-(4-hyd...",Synthesis of N-(diaminomethylidene)-4-[(E)-(4-...,N-(diaminomethylidene)-4-[(E)-(4-hydroxyphenyl...,"1H-NMR (500 MHz, DMSO-d6) δ (ppm): 6.77 (4H, b...","500 MHz, DMSO-d6","6.77, 6.9, 7.8, 7.9","13C-NMR (125 MHz, DMSO-d6) δ: 161.6, 158.1, 15...","125 MHz, DMSO-d6","161.6, 158.1, 153.2, 145.1, 126.8, 125.2, 116.0",N/A,N/A,N/A
2,"{""IUPAC"":""1-(3,4-Dimethylphenyl)-5-phenyl-1H-p...","1-(3,4-Dimethylphenyl)-5-phenyl-1H-pyrazole-3,...","1-(3,4-Dimethylphenyl)-5-phenyl-1H-pyrazole-3,...","1H NMR (400 MHz, CDCl3) δ (ppm) 11.6 (br.s, 2H...","400 MHz, CDCl3","11.6, 8.1, 7.9, 7.6, 7.1, 2.1, 1.8","13C NMR (100 MHz, CDCl3) δ (ppm) 171.3, 167.9 ...","100 MHz, CDCl3","171.3, 167.9, 145.2, 142.1, 135.0, 134.0, 132....",N/A,N/A,N/A
3,"{""IUPAC"":""[(M)-d-4]-C12-TEG"",""1H NMR text"":""1H...",Synthesis of [(M)-d-4]-C12-TEG Under an argon ...,[(M)-d-4]-C12-TEG,"1H NMR (400 MHz, CDCl3): δ 0.86 (9H, t, J = 6....","400 MHz, CDCl3","0.86, 1.25–1.52, 1.72–1.89, 1.97, 2.00, 3.37, ...","13C NMR (100 MHz, CDCl3): δ 14.1, 22.6, 23.2, ...","100 MHz, CDCl3","14.1, 22.6, 23.2, 25.96, 26.02, 28.7, 29.2, 29...",N/A,N/A,N/A
4,"{""IUPAC"":""3-Amino-5-(1-methyl-1H-pyrazol-4-yl)...",4.1 3-Amino-5-(1-methyl-1H-pyrazol-4-yl)pyridi...,3-Amino-5-(1-methyl-1H-pyrazol-4-yl)pyridin-2(...,"1H NMR (500 MHz, DMSO-d6) 3.81 (3H, s), 5.09 (...","500 MHz, DMSO-d6","3.81, 5.09, 6.63, 6.83, 7.59, 7.83, 11.38","13C NMR (126 MHz, DMSO-d6) 157.3 (C), 139.2 (C...","126 MHz, DMSO-d6","157.3, 139.2, 135.6, 127.0, 119.7, 115.8, 112....",N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
295,"{""IUPAC"":""(S)-5-Benzyl 1-tert-butyl 2-(3-((S)-...","(S)-5-Benzyl 1-tert-butyl 2-(3-((S)-1,5-di-te...","(S)-5-Benzyl 1-tert-butyl 2-(3-((S)-1,5-di-ter...","1H NMR (400 MHz, CDCl3) δ 7.33 (m, 5H), 5.10–5...","400 MHz, CDCl3","7.33, 5.10–5.04, 4.38–4.28, 2.51–2.37, 2.32–2....","13C NMR (100 MHz, CDCl3) δ 172.9, 172.5, 172.0...","100 MHz, CDCl3","172.9, 172.5, 172.0, 171.9, 156.8, 135.8, 128....",N/A,N/A,N/A
296,"{""IUPAC"":""Methyl (E)-2-((1H-imidazol-1-yl)meth...",2.1.1. Synthesis of Methyl (E)-2-((1H-imidazol...,Methyl (E)-2-((1H-imidazol-1-yl)methyl)-3-(4-c...,"1H NMR (500MHz, CDCl3) δ 7.96 (s, 1 H), 7.45 (...","500MHz, CDCl3","7.96, 7.45, 7.39, 7.24, 7.01, 6.84, 4.94, 3.80","13C NMR (125 MHz, CDCl3) δ 166.9, 143.7, 137.2...","125 MHz, CDCl3","166.9, 143.7, 137.2, 136.1, 132.5, 130.4, 129....",N/A,N/A,N/A
297,"{""IUPAC"":""N-(2-((4-(3-(3-Hydroxyphenyl)-1-phen...",N-(2-((4-(3-(3-Hydroxyphenyl)-1-phenyl-1H-pyra...,N-(2-((4-(3-(3-Hydroxyphenyl)-1-phenyl-1H-pyra...,"1H NMR (400 MHz, CD3OD) δ 8.05 (s, 1H, Ar-H), ...","400 MHz, CD3OD","8.05, 7.83, 7.76, 7.54–7.50, 7.37–7.34, 7.29–7...","13C NMR (100 MHz, CD3OD) δ 158.6, 157.6, 149.5...","100 MHz, CD3OD","158.6, 157.6, 149.5, 141.9, 140.3, 139.3, 138....",N/A,N/A,N/A
298,"{""IUPAC"":""(1R,4aS)-2-Hydroxy-3-(4-Methoxypheny...","3.2.7. (1R,4aS)-2-Hydroxy-3-(4-Methoxyphenylam...","(1R,4aS)-2-Hydroxy-3-(4-Methoxyphenylamino)Pro...","1H-NMR (600 MHz, DMSO-d6) δ: 7.15 (s, 1H), 6.9...","600 MHz, DMSO-d6","7.15, 6.98, 6.82, 6.67, 6.55, 5.11, 4.07, 3.96...","13C-NMR (150 MHz, DMSO-d6) δ: 177.6, 151.1, 14...","150 MHz, DMSO-d6","177.6, 151.1, 146.7, 145.2, 134.2, 126.5, 124....",N/A,N/A,N/A


### Evaluate

In [2]:
ground_truth_IUPAC = []
ground_truth_1H_NMR = []
ground_truth_1H_NMR_solvent = []
ground_truth_1H_NMR_data = []
ground_truth_13C_NMR = []
ground_truth_13C_NMR_solvent = []
ground_truth_13C_NMR_data = []

predict_IUPAC = []
predict_1HNMR = []
predict_1HNMR_solvent = []
predict_1HNMR_data = []
predict_13CNMR = []
predict_13CNMR_solvent = []
predict_13CNMR_data = []

# ground truth
for i,row in truth_df.iterrows():
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_IUPAC.append(row['IUPAC'])
    ground_truth_1H_NMR.append(row['1H NMR text'])
    ground_truth_1H_NMR_solvent.append(row['1H NMR conditions'])
    ground_truth_1H_NMR_data.append(row['1H NMR data'])
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_13C_NMR_solvent.append(row['13C NMR conditions'])
    ground_truth_13C_NMR_data.append(row['13C NMR data'])

# prediction
for i,row in predict_df.iterrows():
    predict_13CNMR.append(row['13C NMR text'])
    predict_IUPAC.append(row['IUPAC'])
    predict_1HNMR.append(row['1H NMR text'])
    predict_1HNMR_solvent.append(row['1H NMR conditions'])
    predict_1HNMR_data.append(row['1H NMR data'])
    predict_13CNMR.append(row['13C NMR text'])
    predict_13CNMR_solvent.append(row['13C NMR conditions'])
    predict_13CNMR_data.append(row['13C NMR data'])

# Evaluation
print("levenshtein_similarity_of_IUPAC: ", levenshtein_similarity(ground_truth_IUPAC, predict_IUPAC))
print("levenshtein_similarity_of_1HNMR_text: ", levenshtein_similarity(ground_truth_1H_NMR, predict_1HNMR))
print("levenshtein_similarity_of_1HNMR_conditions: ", levenshtein_similarity(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("levenshtein_similarity_of_1HNMR_data: ", levenshtein_similarity(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("levenshtein_similarity_of_13CNMR_text: ", levenshtein_similarity(ground_truth_13C_NMR, predict_13CNMR))
print("levenshtein_similarity_of_13CNMR_conditions: ", levenshtein_similarity(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("levenshtein_similarity_of_13CNMR_data: ", levenshtein_similarity(ground_truth_13C_NMR_data, predict_13CNMR_data))
print()
print("full_sentence_accuracy_of_IUPAC: ", full_sentence_accuracy(ground_truth_IUPAC, predict_IUPAC))
print("full_sentence_accuracy_of_1HNMR_text: ", full_sentence_accuracy(ground_truth_1H_NMR, predict_1HNMR))
print("full_sentence_accuracy_of_1HNMR_conditions: ", full_sentence_accuracy(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("full_sentence_accuracy_of_1HNMR_data: ", full_sentence_accuracy(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("full_sentence_accuracy_of_13CNMR_text: ", full_sentence_accuracy(ground_truth_13C_NMR, predict_13CNMR))
print("full_sentence_accuracy_of_13CNMR_conditions: ", full_sentence_accuracy(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("full_sentence_accuracy_of_13CNMR_data: ", full_sentence_accuracy(ground_truth_13C_NMR_data, predict_13CNMR_data))

levenshtein_similarity_of_IUPAC:  0.9835978612422724
levenshtein_similarity_of_1HNMR_text:  0.9929913505131297
levenshtein_similarity_of_1HNMR_conditions:  0.9880167294342483
levenshtein_similarity_of_1HNMR_data:  0.9956945403219779
levenshtein_similarity_of_13CNMR_text:  0.988710605200284
levenshtein_similarity_of_13CNMR_conditions:  0.9852349586687821
levenshtein_similarity_of_13CNMR_data:  0.9948566883188236

full_sentence_accuracy_of_IUPAC:  0.9466666666666667
full_sentence_accuracy_of_1HNMR_text:  0.8433333333333334
full_sentence_accuracy_of_1HNMR_conditions:  0.9266666666666666
full_sentence_accuracy_of_1HNMR_data:  0.93
full_sentence_accuracy_of_13CNMR_text:  0.8633333333333333
full_sentence_accuracy_of_13CNMR_conditions:  0.9266666666666666
full_sentence_accuracy_of_13CNMR_data:  0.9433333333333334
